In [ ]:
!pip install yfinance

In [3]:
import yfinance as YahooFinance
import pandas as pd
import datetime
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [4]:
eth_information = YahooFinance.Ticker("BTC-USD")
startDate = datetime.datetime(2015, 5, 31)
endDate = datetime.datetime(2024, 1, 30)
df = eth_information.history(start=startDate, end=endDate)

In [5]:
df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2015-05-31 00:00:00+00:00,233.134995,233.251999,229.542007,230.190002,14730800,0.0,0.0
2015-06-01 00:00:00+00:00,230.233002,231.712997,221.296005,222.925995,26090500,0.0,0.0
2015-06-02 00:00:00+00:00,222.893997,226.416000,222.419006,225.802994,20459000,0.0,0.0
2015-06-03 00:00:00+00:00,225.735992,227.404007,223.929993,225.873993,17752400,0.0,0.0
2015-06-04 00:00:00+00:00,225.772003,226.580994,224.054001,224.324005,14728100,0.0,0.0


In [6]:
df = df.drop(["Dividends", "Stock Splits", "Open", "High", "Low"], axis=1)

In [7]:
data = df[["Close"]]
data['prediction'] = (data['Close'] >= df['Close'].shift(15) * 1.05).astype(int)

In [8]:
scaler = MinMaxScaler()

# Scale only the features we need for the LSTM (price, volume)
data[['Close']] = scaler.fit_transform(df[['Close']])

# Define the window length (15 days)
window_length = 15

# Prepare features (price and volume over the last 15 days) and targets (prediction)
def create_sequences(df, window_length):
    X, y = [], []
    for i in range(len(df) - window_length):
        X.append(data[['Close']].iloc[i:i + window_length].values)
        y.append(data['prediction'].iloc[i + window_length])
    return np.array(X), np.array(y)

# Create sequences from the data
X, y = create_sequences(df, window_length)

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Flatten

In [8]:
model = Sequential()

model.add(Dense(units=64, input_shape=(window_length, )))
model.add(Dense(32))
model.add(Dense(16))
model.add(Dense(8))
model.add(Dense(4))
# model.add(Flatten())
model.add(Dense(units=1))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X, y, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5430 - loss: 2.0894 - val_accuracy: 0.7813 - val_loss: 0.5070
Epoch 2/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 672us/step - accuracy: 0.6743 - loss: 0.5802 - val_accuracy: 0.8114 - val_loss: 0.4105
Epoch 3/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 687us/step - accuracy: 0.6915 - loss: 0.6974 - val_accuracy: 0.8637 - val_loss: 0.4197
Epoch 4/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 709us/step - accuracy: 0.7282 - loss: 0.5723 - val_accuracy: 0.7179 - val_loss: 4.5468
Epoch 5/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 672us/step - accuracy: 0.5891 - loss: 1.7863 - val_accuracy: 0.7971 - val_loss: 0.4420
Epoch 6/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 664us/step - accuracy: 0.7071 - loss: 0.5589 - val_accuracy: 0.8700 - val_loss: 0.4509
Epoch 7/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 669us/step - accuracy: 0.7257 - loss: 0.5735 - val_accuracy: 0.8019 - val_loss: 0.3940
Epoch 8/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 684us/step - accuracy: 0.7143 - loss: 0.5348 - val_accuracy: 0.8225 - val_lo

In [9]:
predictions = model.predict(X)
predictions_binary = (predictions > 0.5).astype(int)
tf.saved_model.save(model, 'model')

99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 505us/step
INFO:tensorflow:Assets written to: model/assets


INFO:tensorflow:Assets written to: model/assets


In [40]:
import tf2onnx

In [10]:
!python3 -m tf2onnx.convert --saved-model model --output model.onnx

<frozen runpy>:128: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
2024-09-21 11:12:29,413 - WARNING - '--tag' not specified for saved_model. Using --tag serve
2024-09-21 11:12:29,531 - INFO - Signatures found in model: [serving_default].
2024-09-21 11:12:29,531 - WARNING - '--signature_def' not specified, using first signature: serving_default
2024-09-21 11:12:29,531 - INFO - Output names: ['output_0']
I0000 00:00:1726888349.532480 51779846 devices.cc:76] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
I0000 00:00:1726888349.606256 51779846 devices.cc:76] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2024-09-21 11:12:29,617 - INFO - Using tensorflow=2.17.0, onnx=1.16.2, tf2onnx=1.16.1/

In [ ]:
from verisync import transpile
#verisync

transpile(X[0].flatten().tolist(),"model.onnx")
